# 📘 Mini-Project 2: Scientific Abstracts Retrieval (arXiv)

**Course:** Techniques d'Indexation et de Référencement
**Objective:** Build an IR system using Vector Space Model, TF-IDF, and Rocchio Relevance Feedback.

---

### 1. Imports and Setup
Run this cell to import necessary libraries and download NLTK data

In [ ]:
import pandas as pd
import json
import numpy as np
import os
import re
import nltk
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# Setup NLTK
def download_nltk_dependencies():
    resources = ['stopwords', 'punkt', 'punkt_tab']
    print("Checking NLTK dependencies...")
    for res in resources:
        nltk.download(res, quiet=True)

download_nltk_dependencies()

### 2. Configuration & Constants
Here we define the dataset paths, algorithm parameters, and the test queries with their ground truth categories.

In [ ]:
# ==========================================
# CONFIGURATION
# ==========================================
JSON_FILE = 'arxiv-metadata-oai-snapshot.json'
CSV_FILE = 'arxiv_sample.csv'
INDEX_FILE = 'inverted_index.json'

MAX_ROWS = 10000        # Data limit
TOP_K_RESULTS = 10      # Rank cut-off

# Rocchio Algorithm Weights
ROCCHIO_ALPHA = 1.0     # Original query weight
ROCCHIO_BETA = 0.75     # Relevant docs weight
ROCCHIO_GAMMA = 0.15    # Non-relevant docs weight

# Test Queries
QUERIES = [
    "quantum field theory",
    "semiconductor laser",
    "graph neural network",
    "cosmic microwave background",
    "optical cavity",
    "spintronics",
    "superconducting qubits",
    "photonic integrated circuits"
]

# Relevance Judgment (Ground Truth)
QUERY_CATEGORY_MAP = {
    "quantum field theory": ["hep-th", "quant-ph"],
    "semiconductor laser": ["physics.optics", "cond-mat"],
    "graph neural network": ["cs.LG", "cs.AI", "stat.ML"],
    "cosmic microwave background": ["astro-ph"],
    "optical cavity": ["physics.optics", "quant-ph"],
    "spintronics": ["cond-mat"],
    "superconducting qubits": ["quant-ph"],
    "photonic integrated circuits": ["physics.optics", "eess.IV"]
}

### 3. Data Loading
This step reads the raw JSON dataset (if available) or loads a pre-processed CSV. It combines the `Title` and `Abstract` into a single text column.

In [ ]:
def load_or_create_dataset():
    if os.path.exists(CSV_FILE):
        print(f"Loading existing dataset from {CSV_FILE}...")
        df = pd.read_csv(CSV_FILE)
    else:
        print(f"CSV not found. Generating sample from {JSON_FILE} (Max {MAX_ROWS} rows)...")
        if not os.path.exists(JSON_FILE):
            # Fallback if file doesn't exist (for demo purposes)
            print("⚠️ JSON file not found. Please ensure 'arxiv-metadata-oai-snapshot.json' is in the folder.")
            return pd.DataFrame()
        
        rows = []
        with open(JSON_FILE, 'r') as f:
            for i, line in enumerate(f):
                if i >= MAX_ROWS: 
                    break
                d = json.loads(line)
                rows.append((d['title'], d['abstract'], d['categories']))
        
        df = pd.DataFrame(rows, columns=['title', 'abstract', 'categories'])
        df.to_csv(CSV_FILE, index=False)
        print(f"Dataset saved to {CSV_FILE}")

    # Pre-processing
    df['text'] = (df['title'].fillna('') + '. ' + df['abstract'].fillna(''))
    return df

# Execute Load
df = load_or_create_dataset()
docs = df['text'].tolist()
print(f"Loaded {len(docs)} documents.")
df.head(3)

### 4. Task 1: Inverted Index
We build a manual inverted index to map terms to document IDs.

In [ ]:
def build_inverted_index(docs):
    print("Building Inverted Index...")
    inverted_index = defaultdict(dict)
    stop_words = set(stopwords.words('english'))
    tokenizer = nltk.RegexpTokenizer(r'\w+')

    for doc_id, text in enumerate(docs):
        tokens = tokenizer.tokenize(text.lower())
        for token in tokens:
            if token not in stop_words and len(token) > 2:
                if doc_id not in inverted_index[token]:
                    inverted_index[token][doc_id] = 0
                inverted_index[token][doc_id] += 1
    
    with open(INDEX_FILE, 'w') as f:
        json.dump(inverted_index, f)
    print(f"Inverted index saved to {INDEX_FILE} ({len(inverted_index)} terms)")

# Build the index (Optional: comment out if already exists)
build_inverted_index(docs)

### 5. Vector Space Model (TF-IDF)
We convert the text documents into numerical vectors using Scikit-Learn.

In [ ]:
print("Vectorizing Documents (TF-IDF)...")
# Standard configuration: English stopwords, min document frequency of 3
vec = TfidfVectorizer(stop_words='english', min_df=3)
X = vec.fit_transform(docs)
print(f"Vocabulary size: {len(vec.get_feature_names_out())}")

### 6. Tasks 2, 3 & 4: Search & Evaluation Logic
Definitions for the search engine (Cosine Similarity) and evaluation metrics (P@K, MAP).

In [ ]:
def perform_search(query, vectorizer, X, k=TOP_K_RESULTS):
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, X).ravel()
    top_indices = np.argsort(similarities)[::-1][:k]
    return list(zip(top_indices, similarities[top_indices]))

def is_relevant(doc_categories, query_text):
    target_cats = QUERY_CATEGORY_MAP.get(query_text, [])
    if not target_cats: return False
    doc_cat_list = str(doc_categories).split()
    for t_cat in target_cats:
        if t_cat in doc_cat_list: return True
    return False

def calculate_metrics(results, query, df, k=TOP_K_RESULTS):
    relevant_count = 0
    precisions = []
    
    for rank, (doc_idx, score) in enumerate(results):
        doc_cats = df.iloc[doc_idx]['categories']
        if is_relevant(doc_cats, query):
            relevant_count += 1
            current_precision = relevant_count / (rank + 1)
            precisions.append(current_precision)
            
    p_at_k = relevant_count / k
    ap = 0.0 if relevant_count == 0 else sum(precisions) / relevant_count
    return p_at_k, ap

### 7. Task 5: Rocchio Relevance Feedback
This function refines the query vector based on the initial search results.

In [ ]:
def rocchio_update(query_vec, X, top_indices, doc_categories, query_text):
    relevant_indices = []
    non_relevant_indices = []
    
    for idx in top_indices:
        cats = doc_categories.iloc[idx]
        if is_relevant(cats, query_text):
            relevant_indices.append(idx)
        else:
            non_relevant_indices.append(idx)
            
    q_new = ROCCHIO_ALPHA * query_vec
    
    if relevant_indices:
        Dpos = X[relevant_indices]
        mean_pos = np.array(Dpos.mean(axis=0))
        q_new = q_new + ROCCHIO_BETA * mean_pos
        
    if non_relevant_indices:
        Dneg = X[non_relevant_indices]
        mean_neg = np.array(Dneg.mean(axis=0))
        q_new = q_new - ROCCHIO_GAMMA * mean_neg
        
    q_new[q_new < 0] = 0
    return q_new

### 8. Experiment: Standard Search vs. Rocchio Feedback
We now run the experiment on the 8 queries and compare performance.

In [ ]:
print("\n=== Executing Queries (Standard vs Rocchio) ===")
print(f"{'Query':<30} | {'P@10 (Base)':<12} | {'P@10 (Rocchio)':<12}")
print("-" * 60)

map_base_scores = []
map_rocchio_scores = []

for q in QUERIES:
    # 1. Base Search
    results_base = perform_search(q, vec, X, k=TOP_K_RESULTS)
    p10_base, ap_base = calculate_metrics(results_base, q, df, k=TOP_K_RESULTS)
    map_base_scores.append(ap_base)
    
    # 2. Rocchio Feedback
    q_vec = vec.transform([q])
    top_indices = [idx for idx, score in results_base]
    
    # Compute new query vector
    q_vec_new = rocchio_update(q_vec, X, top_indices, df['categories'], q)
    
    # Search with new vector (manual similarity calc since q_vec_new is already vector)
    sims_new = cosine_similarity(np.asarray(q_vec_new), X).ravel()
    top_indices_new = np.argsort(sims_new)[::-1][:TOP_K_RESULTS]
    results_rocchio = list(zip(top_indices_new, sims_new[top_indices_new]))
    
    # 3. Evaluate Rocchio
    p10_rocchio, ap_rocchio = calculate_metrics(results_rocchio, q, df, k=TOP_K_RESULTS)
    map_rocchio_scores.append(ap_rocchio)
    
    print(f"{q[:30]:<30} | {p10_base:.4f}       | {p10_rocchio:.4f}")

print("-" * 60)
print(f"Mean Average Precision (MAP) Base:    {np.mean(map_base_scores):.4f}")
print(f"Mean Average Precision (MAP) Rocchio: {np.mean(map_rocchio_scores):.4f}")

### 9. Task 6: Ablation Study
Comparing different preprocessing pipelines (Stopwords/Stemming).

In [ ]:
def custom_tokenizer(text):
    stemmer = PorterStemmer()
    tokens = nltk.word_tokenize(text)
    return [stemmer.stem(t) for t in tokens if t.isalnum()]

def run_ablation(df, queries):
    print("\n=== Running Ablation Study (Task 6) ===")
    
    pipelines = [
        ("No Stop, No Stem", None, None),
        ("With Stop, No Stem", 'english', None),
        ("No Stop, With Stem", None, custom_tokenizer),
        ("With Stop, With Stem", 'english', custom_tokenizer)
    ]
    
    results_table = []

    for name, stop_w, tokenizer_func in pipelines:
        print(f"Training Pipeline: {name}...")
        
        vec_ablation = TfidfVectorizer(stop_words=stop_w, tokenizer=tokenizer_func, min_df=3)
        
        try:
            X_ablation = vec_ablation.fit_transform(df['text'])
        except ValueError:
            print(f"Skipping {name}: Empty vocabulary.")
            continue
            
        map_scores = []
        for q in queries:
            results = perform_search(q, vec_ablation, X_ablation, k=TOP_K_RESULTS)
            _, ap = calculate_metrics(results, q, df, k=TOP_K_RESULTS)
            map_scores.append(ap)
            
        mean_map = np.mean(map_scores)
        results_table.append((name, mean_map))
        
    print("\n--- Ablation Results (MAP) ---")
    print(f"{'Pipeline':<25} | {'MAP':<10}")
    print("-" * 37)
    for name, score in results_table:
        print(f"{name:<25} | {score:.4f}")

run_ablation(df, QUERIES)